In [51]:
import numpy as np
import pandas as pd
import pickle as pk

In [52]:
city = 'Perth'

In [53]:
data = pd.read_csv("data_process/_RawData/" + city + "/POIs.csv")
quantity_of_POIS = len(data)
quantity_of_POIS

25

In [55]:
data = pd.read_csv("data_process/_RawData/" + city + "/touristsVisits.csv")
#data = data.head(10)
data.head()


,photoID,userID,dateTaken,poiID,poiTheme,seqID
0,7158915820,11146442@N00,1336155672,1,Structure,1
1,7158914400,11146442@N00,1336156397,7,Park,1
2,7158907712,11146442@N00,1336157902,7,Park,1
3,7158905954,11146442@N00,1336158437,6,Park,1
4,7158899778,11146442@N00,1336170989,23,Entertainment,1


In [56]:
quant = len(data)
quant

3643

In [58]:
global_adj = np.matrix(np.zeros(shape=(quantity_of_POIS,quantity_of_POIS)))
local_adj = dict.fromkeys(list(data['userID']),np.matrix(np.zeros(shape=(quantity_of_POIS,quantity_of_POIS))))
#local_adj

In [59]:
data

,photoID,userID,dateTaken,poiID,poiTheme,seqID
0,7158915820,11146442@N00,1336155672,1,Structure,1
1,7158914400,11146442@N00,1336156397,7,Park,1
2,7158907712,11146442@N00,1336157902,7,Park,1
3,7158905954,11146442@N00,1336158437,6,Park,1
4,7158899778,11146442@N00,1336170989,23,Entertainment,1
...,...,...,...,...,...,...
3638,11476617425,99395734@N08,1383866156,19,Transport,712
3639,10960202885,99395734@N08,1384803458,19,Transport,713
3640,11170645303,99395734@N08,1385810613,23,Entertainment,714
3641,12078439553,99395734@N08,1385960440,19,Transport,715


In [60]:
sequences_dict = {}
biggest_seq = data['seqID'].max()

for seq in range(1, biggest_seq + 1):
    sequence = list(data[data['seqID'] == seq]['poiID'].unique())
    id_user = data[data['seqID'] == seq]['userID'].unique()[0]
    
    if id_user in sequences_dict and len(sequence) > len(sequences_dict[id_user]):
        sequences_dict[id_user] = sequence
        
    elif id_user not in sequences_dict:
        sequences_dict[id_user] = sequence
        
    

In [66]:
#sequences_dict
with open("data_process/" + city +  "/sequence","wb") as file:
    pk.dump(sequences_dict, file)

In [62]:
#data = data.groupby('seqID')
index_i = 0
index_j = 1

while(index_j != quant):
    if(int(data.iloc[[index_i]]['seqID']) == int(data.iloc[[index_j]]['seqID']) and 
       int(data.iloc[[index_i]]['poiID']) != int(data.iloc[[index_j]]['poiID'])):

        user_id = str(data.iloc[[index_i]]['userID'].values[0])
        i = int(data.iloc[[index_i]]['poiID'])-1
        j = int(data.iloc[[index_j]]['poiID'])-1

        local_adj[user_id][i,j] = local_adj[user_id][i,j] + 1
        local_adj[user_id][j,i] = local_adj[user_id][j,i] + 1
        
        global_adj[i,j] = global_adj[i,j] + 1
        global_adj[j,i] = global_adj[j,i] + 1        

    
    index_i = index_i + 1
    index_j = index_j + 1



In [7]:
#print(global_adj)

In [63]:
with open("data_process/local","wb") as file:
    pk.dump(local_adj, file)
with open("data_process/global","wb") as file:
    pk.dump(global_adj, file)

In [64]:
file = open('data_process/global', 'rb')
d = pk.load(file)

In [65]:
d

matrix([[ 0.,  0.,  0.,  1.,  0.,  7., 17.,  0.,  0.,  0.,  0.,  1.,  2.,
          0.,  0.,  0., 17.,  4.,  2.,  7.,  0.,  5.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  8.,  0.,  0.,  2., 17.,
          0.,  0.,  0.,  3.,  1.,  0., 16.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 7.,  1.,  0.,  0.,  0.,  0., 17.,  1.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  7.,  6.,  1.,  1.,  0.,  4.,  1.,  0.,  0.],
        [17.,  1.,  1.,  0.,  0., 17.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  2.,  4.,  4.,  1.,  1., 13.,  1.,  0.